**Summary**

L'objectif de ce TP est de découvrir et de mettre en pratique la Reconnaissance d'Entités Nommées (NER) en utilisant :

- Le dataset CoNLL-2003, un benchmark standard pour la NER.
    - Contient des phrases en anglais avec des entités annotées (Personnes, Organisations, Lieux, Dates, etc.)
    - Format : chaque mot est associé à une étiquette (O, B-PER, I-LOC, etc.)
- La bibliothèque spaCy, un outil puissant pour le traitement du langage naturel (NLP).

La NER est une tâche de NLP qui consiste à identifier et classer des entités nommées dans un texte (par exemple, noms de personnes, organisations, lieux, dates).

spaCy est une bibliothèque open-source pour le traitement du langage naturel (NLP) et inclut des modèles pré-entraînés pour la NER. Elle peut être utilisée pour construire des systèmes d’extraction d’information, de compréhension du langage naturel ou de prétraitement de texte en vue d’un apprentissage approfondi. Cependant, spaCy ne prend pas en charge directement le format CoNLL-2003, donc nous devrons d'abord convertir les données dans un format compatible avec spaCy. Pour plus de détails, vous pouvez consulter cette [vidéo](https://youtu.be/sqDHBH9IjRU).
Le modèle NER de SpaCy est basé sur les CNN (Convolutional Neural Networks).

Vous pouvez explorer d'autres datasets NER comme OntoNotes ou WikiNER.


Il existe des outils d’annotation pour NER comme [Prodigy](https://prodi.gy/) ou autre, mentionné [ici](https://medium.com/dataturks/document-pdf-annotation-tool-f13d94a4b9c). Ces outils permettent de taggers les entités nommées dans chaque séquence.

In [8]:
%%bash

pip install spacy datasets

# Download model
python -m spacy download en_core_web_sm
python -m spacy download en_core_web_lg

# List of available models: https://spacy.io/models

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 103.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.7/400.7 MB 2.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


# Import librairies

In [3]:
%pip install loguru

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 3.0 MB/s eta 0:00:00


In [21]:
from os import environ
from tqdm import tqdm

from loguru import logger
from pathlib import Path
from spacy.tokens import DocBin
from pathlib import Path
from datetime import datetime

# Settings

In [23]:
ROOT_DIR = Path.cwd().parent  # project folder
DATA_DIR = Path(ROOT_DIR, "data")  # data
CONFIG_DIR = Path(ROOT_DIR, "config")  # config, bins
OUTPUT_DIR  = Path(ROOT_DIR, "ner_output")  # models

DATA_DIR.mkdir(parents=True, exist_ok=True)
CONFIG_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Add paths in env
environ["SPACY_BINS_DIR"] = str(CONFIG_DIR)
environ["SPACY_DATA_DIR"] = str(DATA_DIR)
environ["SPACY_OUTPUT_DIR"] = str(OUTPUT_DIR)

logger.info(f"\nRoot directory: {ROOT_DIR} \nData directory: {DATA_DIR} \nConfig directory: {CONFIG_DIR} \nOutput directory: {OUTPUT_DIR}")

2026-02-12 23:36:11.012 | INFO     | __main__:<cell line: 0>:15 - 
Root directory: / 
Data directory: /data 
Config directory: /config 
Output directory: /ner_output


# Use spaCy to make a NER example

### Test pre-trained model

In [7]:
import spacy

# Load model
nlp = spacy.load("en_core_web_sm")

# Text example
text = "Apple is looking at buying U.K. startup for $1 billion in 2023."

# Text preprocessing
doc = nlp(text)

# Display entity
for ent in doc.ents:
    print(f"{ent.text} -> {ent.label_}")

Apple -> ORG
U.K. -> GPE
$1 billion -> MONEY
2023 -> DATE


In [7]:
spacy.displacy.serve(doc, style="ent", auto_select_port=True)

/usr/local/lib/python3.12/dist-packages/spacy/displacy/__init__.py:108: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  warnings.warn(Warnings.W011)



Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


### SpaCy QickStart

https://spacy.io/usage/training#quickstart

# Train a custom NER

## Load data

Le jeu de données CoNLL-2003, très connu pour les tâches de Reconnaissance d'Entités Nommées (NER) comme PERSON, LOCATION, ORGANIZATION, etc;

Le dataset CoNLL-2003 est structuré comme suit :

- Chaque phrase est représentée par une séquence de tokens.
- Chaque token est associé à une étiquette NER au format IOB (Inside, Outside, Beginning).
    - `B-PER` : Début d'une entité de type "Personne".
    - `I-PER` : Suite d'une entité de type "Personne".
    - `B-ORG` : Début d'une entité de type "Organisation".
    - `I-ORG` : Suite d'une entité de type "Organisation".
    - `B-LOC` : Début d'une entité de type "Lieu".
    - `I-LOC` : Suite d'une entité de type "Lieu".
    - `B-MISC` : Début d'une entité de type "Divers".
    - `I-MISC` : Suite d'une entité de type "Divers".
    - `O` : Token ne faisant pas partie d'une entité.

In [11]:
%pip uninstall -y datasets

Found existing installation: datasets 4.0.0
Uninstalling datasets-4.0.0:
  Successfully uninstalled datasets-4.0.0


In [1]:
%pip install datasets==1.18.4

In [9]:
from datasets import load_dataset
dataset_conll = load_dataset("conll2003")
logger.info(f"\n {dataset_conll['train'][0]}")

  0%|          | 0/3 [00:00<?, ?it/s]

2026-02-12 23:31:53.609 | INFO     | __main__:<cell line: 0>:3 - 
 {'id': '0', 'tokens': ['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.'], 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7], 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0], 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}


In [11]:
label_list = dataset_conll["train"].features["ner_tags"].feature.names
logger.info(f"NER labels: {label_list}")

2026-02-12 23:31:54.246 | INFO     | __main__:<cell line: 0>:2 - NER labels: ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']


In [12]:
# Example
dataset_conll["train"][0]

{'id': '0',
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}

## Modeling

### Preprocessing

- Conversion des données en objets Doc et DocBin, le format interne de spaCy.
- Reconstruction des entités à partir des tags IOB (Begin, Inside, Outside).

In [13]:
def create_spacy_docs(dataset_split, label_names):
    """Convert data to SpaCy format."""
    db = DocBin()  # doc binary
    for example in tqdm(dataset_split, desc="Doc binary transformation ..."):
        words = example['tokens']
        labels = example['ner_tags']
        doc = nlp.make_doc(" ".join(words))
        ents = []
        start = 0
        for word, label_idx in zip(words, labels):
            label = label_names[label_idx]
            word_start = doc.text.find(word, start)
            word_end = word_start + len(word)
            if label != "O":
                if label.startswith("B-") or label.startswith("I-"):
                    ents.append(doc.char_span(word_start, word_end, label=label[2:]))
            start = word_end
        ents = [e for e in ents if e is not None]
        doc.ents = ents
        db.add(doc)
    return db

In [14]:
# Create files
train_db = create_spacy_docs(dataset_conll["train"], label_names=label_list)
valid_db = create_spacy_docs(dataset_conll["validation"], label_names=label_list)
test_db = create_spacy_docs(dataset_conll["test"], label_names=label_list)


train_db.to_disk(Path(DATA_DIR, "conll_train.spacy"))
valid_db.to_disk(Path(DATA_DIR, "conll_valid.spacy"))
test_db.to_disk(Path(DATA_DIR, "conll_test.spacy"))

Doc binary transformation ...: 100%|██████████| 3454/3454 [00:02<00:00, 1591.16it/s]


spaCy utilise un fichier de configuration pour définir les paramètres d'entraînement. Vous pouvez générer un fichier de configuration de base avec la commande suivante :

### SpaCy config file
<img src="attachment:afb1d94c-f254-4a3f-90c0-70d5e6010102.png" alt="spaCy" style="width: 400px;"/>

le fichier de configuration (`config.cfg`) est un élément indispensable qui définit tous les **composants**, **hyperparamètres** et **pipelines** nécessaires à l’entraînement ou à l’utilisation d’un modèle NLP.

**À quoi sert ce fichier ?**
- Définir les composants du pipeline (tokenizer, tagger, parser, NER, etc.)
- Spécifier les données d'entraînement et de validation
- Définir les modèles de embeddings (par exemple, tok2vec)
- Choisir les optimiseurs, stratégies d’apprentissage, batch sizes, etc.
- Rendre le processus reproductible.

#### spacy init config

La commande spacy init config est utilisée pour générer un fichier de configuration de base pour un projet spaCy. Ce fichier de configuration définit les paramètres d'entraînement, les composants du pipeline, les hyperparamètres, etc.

`python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency`

Options principales :
- `lang` : spécifie la langue du modèle (par exemple, `en` pour l'anglais).
- `pipeline` : définit les composants du pipeline (par exemple, `ner` pour la reconnaissance d'entités nommées).
- `optimize` : spécifie l'optimisation pour `efficiency` (efficacité) ou `accuracy` (précision).

In [15]:
%%bash
# Generate config file got spaCy
python -m spacy init config ${SPACY_BINS_DIR}/config.cfg --lang en --pipeline ner --optimize efficiency --force

⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
/config/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


#### spacy init fill-config

La commande spacy init fill-config est utilisée pour compléter un fichier de configuration existant avec des valeurs par défaut pour les paramètres manquants. Elle est utile lorsque vous avez un fichier de configuration partiel: `base_config`. Ce dernier vous pouvez le télécharger [ici](https://spacy.io/usage/training#quickstart).

Un fichier, nommé `acc_config.cfg`, est généré en ajoutant les valeurs par défaut aux paramètres manquants dans `base_config.cfg`. Le nom du fichier peut être personnalisé selon vos préférences. Par exemple, j’ai ajouté le préfixe `acc_` (puisque le fichier `base_config.cfg` a été téléchargé avec l’option `accuracy`) afin de le distinguer de celui généré précédemment avec l’option `efficiency`.

In [26]:
%%bash
python -m spacy init fill-config ${SPACY_BINS_DIR}/acc_base_config.cfg ${SPACY_BINS_DIR}/acc_config.cfg

✔ Auto-filled config with all values
✔ Saved config
/config/acc_config.cfg
You can now add your data and train your pipeline:
python -m spacy train acc_config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


### Training

In [27]:
%%bash

START=$(date '+%Y-%m-%d %H:%M:%S')
echo "Start time: $START"
printf "\nConfig file path: $SPACY_BINS_DIR \nData directory: ${SPACY_DATA_DIR}"

# scan/debug data: vérifier la qualité, la structure et la compatibilité de vos données d'entraînement et de validation
printf "**************************************** Starting debug config...\n\n"
python -m spacy debug data $SPACY_BINS_DIR/acc_config.cfg --paths.train $SPACY_DATA_DIR/conll_train.spacy --paths.dev $SPACY_DATA_DIR/conll_valid.spacy

Start time: 2026-02-12 23:42:18

Config file path: /config 
Data directory: /data**************************************** Starting debug config...


============================ Data file validation ============================
✔ Pipeline can be initialized with data
✔ Corpus is loadable

=============================== Training stats ===============================
Language: fr
Training pipeline:
14041 training docs
3250 evaluation docs
⚠ 129 training examples also in evaluation data

============================== Vocab & Vectors ==============================
ℹ 216122 total word(s) in the data (22303 unique)
⚠ 409 misaligned tokens in the training data
⚠ 132 misaligned tokens in the dev data
ℹ No word vectors present in the package

================================== Summary ==================================
✔ 2 checks passed
⚠ 3 warnings


In [28]:
%%bash

START=$(date '+%Y-%m-%d %H:%M:%S')
echo "Start time: $START"

printf "\nConfig file path: $SPACY_BINS_DIR \nData directory: ${SPACY_DATA_DIR} \nOutput directory: ${SPACY_OUTPUT_DIR}\n\n"

printf "\n\n**************************************** Starting training model...\n\n"

python -m spacy train $SPACY_BINS_DIR/acc_config.cfg \
--paths.train $SPACY_DATA_DIR/conll_train.spacy --paths.dev $SPACY_DATA_DIR/conll_valid.spacy \
--output $SPACY_OUTPUT_DIR

Start time: 2026-02-12 23:42:36

Config file path: /config 
Data directory: /data 
Output directory: /ner_output



**************************************** Starting training model...

ℹ Saving to output directory: /ner_output
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: []
ℹ Initial learn rate: 0.001
E    #       SCORE 
---  ------  ------
  0       0    0.00
  0     200    0.00
  0     400    0.00
  0     600    0.00
  0     800    0.00
  0    1000    0.00
  0    1200    0.00
  1    1400    0.00
  1    1600    0.00
✔ Saved pipeline to output directory
/ner_output/model-last


- `E`: Numéro de l’époque (epoch) en cours
- `#`: Numéro du lot (batch) (par exemple: à l'étape initiale 0: 100 séquences ont été utilisées, cf [training.batcher.size] dans le fichier de config)
- `LOSS TOK2VEC` : Perte associée à la représentation des tokens
- `LOSS NER`: Perte associée à la prédiction des entités nommées
- `ENTS_F`: Score F1 sur les entités (moyenne harmonique de la précision et du rappel)
- `ENTS_P`: précision score (mesure la fraction des entités prédites par le modèle qui sont correctes)
- `ENTS_R`: recall score (mesure la fraction des entités réelles qui sont correctement détectées par le modèle)
- `SCORE`: score global du modèle, qui est généralement identique au score F1 (ENTS_F) pour la tâche de NER

### Test

In [29]:
%%bash

# performance evaluation
START=$(date '+%Y-%m-%d %H:%M:%S')
echo "Start time: $START"

printf "\n\n***************************************** Training performances\n"
python -m spacy evaluate ${SPACY_OUTPUT_DIR}/model-best $SPACY_DATA_DIR/conll_train.spacy --output ${SPACY_OUTPUT_DIR}/train_evaluation.json

printf "\n\n***************************************** Validation performances\n"
python -m spacy evaluate ${SPACY_OUTPUT_DIR}/model-best $SPACY_DATA_DIR/conll_valid.spacy --output ${SPACY_OUTPUT_DIR}/valid_evaluation.json

printf "\n\n***************************************** Test performances\n"
python -m spacy evaluate ${SPACY_OUTPUT_DIR}/model-best $SPACY_DATA_DIR/conll_test.spacy --output ${SPACY_OUTPUT_DIR}/test_evaluation.json

END=$(date '+%Y-%m-%d %H:%M:%S')
echo "End time: $END"

Start time: 2026-02-12 23:43:08


***************************************** Training performances
ℹ Using CPU

================================== Results ==================================

TOK     99.81 
SPEED   158084

✔ Saved results to /ner_output/train_evaluation.json


***************************************** Validation performances
ℹ Using CPU

================================== Results ==================================

TOK     99.76 
SPEED   479802

✔ Saved results to /ner_output/valid_evaluation.json


***************************************** Test performances
ℹ Using CPU

================================== Results ==================================

TOK     99.79 
SPEED   466220

✔ Saved results to /ner_output/test_evaluation.json
End time: 2026-02-12 23:43:44


### Prediction

In [30]:
custom_nlp_ner = spacy.load(Path(environ["SPACY_OUTPUT_DIR"], "model-best"))

text_test = "Barack Obama visited London in 2020."

doc_test = custom_nlp_ner(text_test)
for ent in doc_test.ents:
    print(ent.text, ent.label_)

In [33]:
spacy.displacy.serve(doc_test, style="ent", auto_select_port=True)

/usr/local/lib/python3.12/dist-packages/spacy/displacy/__init__.py:108: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  warnings.warn(Warnings.W011)
/usr/local/lib/python3.12/dist-packages/spacy/displacy/__init__.py:215: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)



Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


## Comment optimiser un modèle NER

- **1. Optimiser les données**
    - Qualité des annotations
    - Volume de données
    - Diversité
- **Ajuster la configuration (config.cfg)**
    - Taille du réseau (hidden_width) : par défaut 64 pour certains modèles. Tester 128 ou 256 pour un modèle plus puissant.
    - Utilisation de vecteurs pré-entraînés : spécifier un vecteur comme en_core_web_lg ou des vecteurs custom
    - Dropout : pour éviter le surapprentissage
    - Augmenter max_steps ou max_epochs
- **Stratégies d’entraînement**
    - Évaluation fréquente: `eval_frequency = 200` permet de sauvegarder le meilleur modèle plus souvent.
    - Batching intelligent utiliser un batching compounding
- **4. Transfert de connaissance: utiliser un tok2vec pré-entraîné**
- **6. Data augmentation (facultatif mais puissant)**
     - Ajouter des variantes dans les textes : paraphrases, substitutions d’entités par d’autres, ajout de synonymes, ...
     - Vous pouvez utiliser des outils comme `nlpaug`, `textattack`, etc.

### **TO DO**



1.   Indexer les noms de vos modèles avec votre **nom_prenom**
2.   Fichier de performances (ex: train_evaluation.json), vous les renommez en

  - <YYYYMMDD_train_evaluation.json>  
- <YYYYMMDD_valid_evaluation.json>
- <YYYYMMDD_test_evaluation.json>  



YYYYMMDD : année (YYYY), mois (MM) et jour (DD)



In [31]:
# --- 1. Indexer les noms de vos modèles avec votre nom_prenom ---
MY_NAME_PRENOM = "diop_papamagatte"

output_dir = Path(environ["SPACY_OUTPUT_DIR"])
model_best_path = output_dir / "model-best"
new_model_name = f"{MY_NAME_PRENOM}-model-best"
new_model_path = output_dir / new_model_name

if model_best_path.exists() and not new_model_path.exists():
    model_best_path.rename(new_model_path)
    print(f"Modèle renommé de '{model_best_path.name}' à '{new_model_path.name}'")
elif new_model_path.exists():
    print(f"Le modèle '{new_model_path.name}' existe déjà. Pas de renommage nécessaire.")
else:
    print(f"Le répertoire '{model_best_path.name}' n'existe pas.")



Modèle renommé de 'model-best' à 'diop_papamagatte-model-best'


In [32]:
# --- 2. Renommer les fichiers de performances avec YYYYMMDD ---
current_date = datetime.now().strftime("%Y%m%d")

evaluation_files = [
    "train_evaluation.json",
    "valid_evaluation.json",
    "test_evaluation.json",
]

for filename in evaluation_files:
    old_path = output_dir / filename
    if old_path.exists():
        new_filename = f"{current_date}_{filename}"
        new_path = output_dir / new_filename
        old_path.rename(new_path)
        print(f"Fichier de performance renommé de '{filename}' à '{new_filename}'")
    else:
        print(f"Le fichier '{filename}' n'existe pas dans '{output_dir}'.")

Fichier de performance renommé de 'train_evaluation.json' à '20260212_train_evaluation.json'
Fichier de performance renommé de 'valid_evaluation.json' à '20260212_valid_evaluation.json'
Fichier de performance renommé de 'test_evaluation.json' à '20260212_test_evaluation.json'
